# 26-쇼핑몰 장바구니 데이터 수집
## Selenium 소개
Selenium은 주로 웹앱을 테스트하는 웹 프레임워크로서 크롬이나 파이어폭스같은 각 브라우저의 각 브라우저마다 제공되는 Webdriver API를 활용하여 크롬이나 파이어폭스 같은 웹 브라우저를 Java나 C#, Python 같은 프로그래밍 언어를 통해 제어합니다.

웹 브라우저를 직접 제어하기 때문에 자바스크립트에 의해 동적으로 생성되는 사이트의 데이터를 크롤링할 때 매우 유용하게 사용되는 스크래핑 도구입니다.

### 1) Selenium 동작 방식
Selenium은 아래와 같은 과정을 거쳐 크롬 브라우저를 제어합니다.

Python 소스코드 --> Selenium 패키지 --> ChromeDriver(WebDriver) --> Google Chrome

키보드 입력값을 전달한다거나 스크롤을 이동시키는 등의 제어가 가능하고 웹 브라우저의 개발자도구에서 확인 가능한 Element 내용을 수집하기 때문에 웹 페이지 로딩 후 동적인 결과를 가져오는 웹 페이지의 컨텐츠를 수집할 수 있습니다. requests 패키지를 통해 URL을 수집하는 경우 동적 결과는 가져오지 못하는 경우 Selenium을 활용하면 컨텐츠 수집이 가능합니다.

### 2) ChromeDriver 내려받기
원래는 https://chromedriver.chromium.org/downloads에서 사용중인 운영체제와 Google Chrome 브라우저의 버전에 맞는 ChromeDriver를 내려받아서 python 소스코드에서 접근할 수 있는 경로상에 ChromeDriver를 배치해야 한다.

크롬 특성상 버전업이 자주 있기 때문에 매번 ChromeDriver를 직접 내려받는 번거로움이 발생.

chromedriver_autoinstaller 패키지를 사용하면 Python 스스로 현재 크롬 버전에 맞는 ChromeDriver를 내려받아 사용할 수 있다.

### 3) selenium 설치하기
- pip install --upgrade selenium
- pip install --upgrade chromedriver_autoinstaller

In [5]:
# ChromeDriver 자동 설치 모듈
import chromedriver_autoinstaller
# Chrome을 제어하기 위한 객체
from selenium import webdriver
# Chrome이 웹 페이지 로딩을 완료 할 때까지 최대 n초간 대기하는 기능.
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
from pandas import DataFrame
# 파이썬 프로그램에 지정된 시간동안 랙을 거는 기능을 위해 사용
import time

In [6]:
chromedriver_autoinstaller.install()

'c:\\users\\ezen\\appdata\\local\\programs\\python\\python38\\lib\\site-packages\\chromedriver_autoinstaller\\90\\chromedriver.exe'

In [7]:
# 크롬드라이버 실행
driver = webdriver.Chrome()

# 크롬브라우저가 준비될 때 까지 최대 5초씩 대기
driver.implicitly_wait(5)

## 샘플 쇼핑몰 로그인
### 1) 로그인 페이지로 이동하기

In [19]:
driver.get('http://itproject.ezenac.co.kr/springmyshop/account/login')
time.sleep(3)

### 2) 로그인하기

In [20]:
id_input = WebDriverWait(driver,3).until(lambda x: x.find_element_by_css_selector('#user_id'))

id_input.send_keys('cj5622')

In [21]:
pw_input = WebDriverWait(driver, 3).until(lambda x: x.find_element_by_css_selector("#user_pw"))
pw_input.send_keys("sl237&46")

In [22]:
login_button = WebDriverWait(driver, 3).until(
                lambda x: x.find_element_by_css_selector("button[type='submit']"))
login_button.click()

In [23]:
time.sleep(3)

## 장바구니 데이터 수집
### 1) 장바구니 페이지로 이동

In [24]:
driver.get("http://itproject.ezenac.co.kr/springmyshop/shopping/cart")
# 페이지를 이동하는 동안 최대 3초간 대기
time.sleep(3)

### 2) 장바구니 페이지의 HTML 코드를 BeautifulSoup 객체로 생성

#### HTML 코드 추출

In [25]:
html = driver.page_source
#print(html)

#### BeautifulSoup 객체로 변환

In [26]:
soup = BeautifulSoup(html, 'html.parser')

In [27]:
#크롬 브라우저 닫기
driver.quit()

### 3) BeautifulSoup을 활용하여 장바구니 데이터 가져오기
장바구니 내역 가져오기

In [28]:
tr = soup.select('.cart-item')
tr

[<tr class="cart-item cart-item-935">
 <td class="text-center" rowspan="1">
 <input checked="" class="cart_id" name="cart_id[]" type="checkbox" value="935"/>
 </td>
 <td class="text-center" rowspan="1">
 <a href="/springmyshop/product/detail/2175">
 <img src="" width="50"/>
 </a>
 </td>
 <td class="text-center" rowspan="1">
 <a href="/springmyshop/product/detail/2175">코데모 셔링원피스</a>
 </td>
 <td class="text-center">블랙(black)</td>
 <td class="text-center">
 <span class="price" data-value="35800">
                                                 35,800</span>
 <span>원</span>
 </td>
 <td class="text-center">1</td>
 <td class="text-center text-primary">
 <span class="total" data-value="35800">
                                                 35,800</span>
 <span>원</span>
 </td>
 <td class="text-center">
 <strong class="text-warning"><span class="order-price" data-value="35800">
                                                         35,800</span>원</strong>
 </td>
 <td class="text-center">
 

In [30]:
cart = []

for tr_item in tr:
    td = tr_item.select('.text-center')
    
    item_dict = {
        '상품명' : td[2].text.strip(),
        '옵션' : td[3].text.strip(),
        '상품가격' : int(td[4].text.strip().replace('원',"").replace(',','')),
        "수량": int(td[5].text.strip()),
        "합계": int(td[6].text.strip().replace("원", "").replace(",", "")),
        "판매금액": int(td[7].text.strip().replace("원", "").replace(",", ""))
    }
    
    cart.append(item_dict)
    
cart

[{'상품명': '코데모 셔링원피스',
  '옵션': '블랙(black)',
  '상품가격': 35800,
  '수량': 1,
  '합계': 35800,
  '판매금액': 35800},
 {'상품명': '카이덴 캉캉스커트',
  '옵션': '블랙-도트(black)',
  '상품가격': 19000,
  '수량': 1,
  '합계': 19000,
  '판매금액': 19000},
 {'상품명': '엔티스운동화',
  '옵션': '화이트(white), 250',
  '상품가격': 26900,
  '수량': 1,
  '합계': 26900,
  '판매금액': 26900},
 {'상품명': '티트랑 미키bag',
  '옵션': '레드(red)',
  '상품가격': 44000,
  '수량': 1,
  '합계': 44000,
  '판매금액': 44000}]

In [31]:
cart_df = DataFrame(cart)
cart_df

,상품명,옵션,상품가격,수량,합계,판매금액
0,코데모 셔링원피스,블랙(black),35800,1,35800,35800
1,카이덴 캉캉스커트,블랙-도트(black),19000,1,19000,19000
2,엔티스운동화,"화이트(white), 250",26900,1,26900,26900
3,티트랑 미키bag,레드(red),44000,1,44000,44000


In [32]:
cart_df['판매금액'].sum()

125700